# Code for implementing the traditional Huff and the time-aware dynamic Huff model (T-Huff)

### Reference: Liang, Y., Gao, S., Cai, Y., Foutz, N. Z., & Wu, L. (2020). Calibrating the dynamic Huff model for business analysis using location big data. Transactions in GIS. 24(3), 1-23. DOI: 10.1111/tgis.12624. arXiv preprint available at https://arxiv.org/abs/2003.10857.

### Data: SafeGraph (https://shop.safegraph.com/)

<img src="https://geods.geography.wisc.edu/wp-content/uploads/2020/04/tgis12624_prediction.jpg"
     alt="Preiction vs. Actual"
     style="float: left; margin-right: 10px;" />

# import the pyswarms lib

In [1]:
import sys
!{sys.executable} -m pip install pyswarms

     |████████████████████████████████| 102kB 4.0MB/s 


In [0]:
# Import modules
import numpy as np

# Import PySwarms
#import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
#%load_ext autoreload
#%autoreload 2

# define the Huff model

In [0]:
def HuffModel(mx,alpha,beta,att_field):
    #get the sum of H (dinominator)
    #print(alpha)
    #print(beta)
    cbg_sum = {}
    for i,row in mx.iterrows():
        if mx.loc[i]["census_block_group"] not in cbg_sum.keys():
            A = mx.loc[i][att_field]
            D = mx.loc[i]["distance"]
            cbg_sum[mx.loc[i]["census_block_group"]] = HuffPro(A,D,alpha,beta)
        else:
            A = mx.loc[i][att_field]
            D = mx.loc[i]["distance"]
            cbg_sum[mx.loc[i]["census_block_group"]] += HuffPro(A,D,alpha,beta)
    mx["pro"] = ""
    mx["h_sum"] = ""
    for i,row in mx.iterrows():
        mx.loc[i,"h_sum"] = cbg_sum[mx.loc[i]["census_block_group"]]
        A = mx.loc[i][att_field]
        D = mx.loc[i]["distance"]
        mx.loc[i,"pro"] =  HuffPro(A,D,alpha,beta)/mx.loc[i]["h_sum"]
    return mx
    

In [0]:
def HuffPro(A,D,alpha,beta):
    return pow(A,alpha)/pow(D,beta)

In [0]:
#"Whole Foods Market"
def Corr(mx,alpha,beta,att_field,location_name):
    mx_hf = HuffModel(mx,alpha,beta,att_field)
    mx_hf["pro"] = mx_hf["pro"].astype(float)
    mx_hf["actual_pro"] = mx_hf["actual_pro"].astype(float)
    mx_hf_location = mx_hf[mx_hf["location_name"] == location_name]
    cor = mx_hf_location[["actual_pro","pro"]].corr().values[0][1]
    #cor_total = mx_hf[["actual_pro","pro"]].corr().values[0][1]
    return mx_hf, cor

In [0]:
def THuffModelHourlyLocation(mx,alpha,beta,att_field,location_name,hourlyfield):
    cbg_sum = {}
    for i,row in mx.iterrows():
        if mx.loc[i]["census_block_group"] not in cbg_sum.keys():
            A = mx.loc[i][att_field]
            D = mx.loc[i]["distance"]
            cbg_sum[mx.loc[i]["census_block_group"]] = HuffPro(A,D,alpha,beta)
        else:
            A = mx.loc[i][att_field]
            D = mx.loc[i]["distance"]
            cbg_sum[mx.loc[i]["census_block_group"]] += HuffPro(A,D,alpha,beta)
    mx_location = mx[mx["location_name"] == location_name]
    mx_location["pro_total"] = ""
    mx_location["h_sum"] = ""
    mx_location["pro_hourly_est"] = ""
    mx_location["pro_hourly_actual"] = ""
    for i,row in mx_location.iterrows():
        mx_location.loc[i,"h_sum"] = cbg_sum[mx_location.loc[i]["census_block_group"]]
        A = mx_location.loc[i][att_field]
        D = mx_location.loc[i]["distance"]
        mx_location.loc[i,"pro_total"] =  HuffPro(A,D,alpha,beta)/mx_location.loc[i]["h_sum"]
        tem_dic_est = {}
        tem_dic_act = {}
        for key in mx_location.loc[i][hourlyfield].keys():
            tem_dic_est[key] = [x*mx_location.loc[i]["pro_total"] for x in mx_location.loc[i][hourlyfield][key]]
            tem_dic_act[key] = [x*mx_location.loc[i]["actual_pro"] for x in mx_location.loc[i][hourlyfield][key]]
        mx_location.loc[i,"pro_hourly_est"] = [tem_dic_est]
        mx_location.loc[i,"pro_hourly_actual"] = [tem_dic_act]
    return mx_location

In [0]:
def CorrHourlyLocation(mx,alpha,beta,att_field,location_name,hourlyfield,actual_field,est_field):
    mx_hf_location = THuffModelHourlyLocation(mx,alpha,beta,att_field,location_name,hourlyfield)
    #print(mx_hf)
    actual_list = []
    est_list = []
    for i,row in mx_hf_location.iterrows():
        actual_list.extend(mx_hf_location.loc[i][actual_field]['Monday'])
        actual_list.extend(mx_hf_location.loc[i][actual_field]['Tuesday'])
        actual_list.extend(mx_hf_location.loc[i][actual_field]['Wednesday'])
        actual_list.extend(mx_hf_location.loc[i][actual_field]['Thursday'])
        actual_list.extend(mx_hf_location.loc[i][actual_field]['Friday'])
        actual_list.extend(mx_hf_location.loc[i][actual_field]['Saturday'])
        actual_list.extend(mx_hf_location.loc[i][actual_field]['Sunday'])
        est_list.extend(mx_hf_location.loc[i][est_field]['Monday'])
        est_list.extend(mx_hf_location.loc[i][est_field]['Tuesday'])
        est_list.extend(mx_hf_location.loc[i][est_field]['Wednesday'])
        est_list.extend(mx_hf_location.loc[i][est_field]['Thursday'])
        est_list.extend(mx_hf_location.loc[i][est_field]['Friday'])
        est_list.extend(mx_hf_location.loc[i][est_field]['Saturday'])
        est_list.extend(mx_hf_location.loc[i][est_field]['Sunday'])
        #print(actual_list)
        #print(est_list)
        cor = numpy.corrcoef(actual_list,est_list)[0][1]
    return mx_hf_location, cor

# Read the data we processed. More raw data can be found from SafeGraph Inc. 

In [0]:
import pandas as pd
import numpy
infile = "la_wholefoods.csv"
mx = pd.read_csv(infile,dtype={'census_block_group': str})
mx["raw_visit_counts"] = mx["raw_visit_counts"].astype(float)
mx["distance"] = mx["distance"].astype(float)
import ast
for i,row in mx.iterrows():
    mx.at[i,"popularity_day_pro"] = ast.literal_eval(mx.iloc[i]["popularity_day_pro"])
    mx.at[i,"popularity_hourly_pro"] = ast.literal_eval(mx.iloc[i]["popularity_hourly_pro"])

# Traditonal Huff model

## can input alpha, beta directly to get the correlation.

In [0]:
alpha_list_ori_detailed = [0.1,0.5,1,2,5]
beta_list = [0.1,0.5,1,2,5]
result = {}
for alpha in alpha_list_ori_detailed:
    for beta in beta_list:
        att_field = "raw_visit_counts"
        location_name = "Whole Foods Market" 
        mx_hf,cor = Corr(mx,alpha,beta,att_field,location_name)
        result[alpha,beta] = cor
        print(alpha, ",",beta,": ",cor)

## optimization to find the alpha, beta with a range (0,2)

In [0]:
def PSOfunction_Huff(x): 
    print(x)
    import timeit
    #start_time = timeit.default_timer()
    alpha = x[:,0]
    beta = x[:,1]
    i = 0
    max_cor = -9999
    att_field = "raw_visit_counts"
    location_name = "Whole Foods Market"
    n_particles = x.shape[0]
    j = []
    for i in range(n_particles):
        mx_hf,cor = Corr(mx,alpha[i],beta[i],att_field,location_name)
        j.append(-cor)
    print(j)
    return np.array(j)

In [0]:
from pyswarms.single.global_best import GlobalBestPSO

#using Whole Foods with 
x_max = 2* np.ones(2)
x_min = 0* np.ones(2)
bounds = (x_min, x_max)
print(bounds)
options = {'c1': 2, 'c2': 2, 'w': 0.9}
#init_pos = pos
optimizer = GlobalBestPSO(n_particles=10, dimensions=2,options=options, bounds=bounds)

#kwargs={"mx": mx}
cost, pos = optimizer.optimize(PSOfunction_Huff, 10)

In [0]:
cost,pos

# T-Huff model

## considering the temporal visit changes
<img src="https://geods.geography.wisc.edu/wp-content/uploads/2020/04/tgis12624_HourlyPlot.png"
     alt="Preiction vs. Actual"
     style="float: left; margin-right: 10px;" />

## can input alpha, beta directly to get the correlation.

In [0]:
alpha_list_ori_detailed = [0.1,0.5,1,2,5]
beta_list = [0.1,0.5,1,2,5]
result_dic_ori_detailed_hourly = {}
for alpha in alpha_list_ori_detailed:
    for beta in beta_list:
        att_field = "raw_visit_counts"
        location_name = "Whole Foods Market" 
        mx_hf,cor = CorrHourlyLocation(mx,alpha,beta,att_field,location_name,"popularity_hourly_pro",'pro_hourly_actual','pro_hourly_est')
        result_dic_ori_detailed_hourly[alpha,beta] = cor
        print(alpha, ",",beta,": ",cor)

## can also just use optimation to get optimal alpha, beta

In [0]:
def PSOfunction_THuff(x): 
    print(x)
    import timeit
    #start_time = timeit.default_timer()
    alpha = x[:,0]
    beta = x[:,1]
    i = 0
    max_cor = -9999
    att_field = "raw_visit_counts"
    location_name = "Whole Foods Market"
    n_particles = x.shape[0]
    j = []
    for i in range(n_particles):
        mx_hf,cor = CorrHourlyLocation(mx,alpha[i],beta[i],att_field,location_name,"popularity_hourly_pro",'pro_hourly_actual','pro_hourly_est')
        j.append(-cor)
    print(j)
    return np.array(j)

In [0]:
from pyswarms.single.global_best import GlobalBestPSO

#using Whole Foods with 
x_max = 2* np.ones(2)
x_min = 0* np.ones(2)
bounds = (x_min, x_max)
print(bounds)
options = {'c1': 2, 'c2': 2, 'w': 0.9}
#init_pos = pos
optimizer = GlobalBestPSO(n_particles=10, dimensions=2,options=options, bounds=bounds)

#kwargs={"mx": mx}
cost, pos = optimizer.optimize(PSOfunction_THuff, 10)

(array([0., 0.]), array([2., 2.]))


In [0]:
cost, pos

## output the data for mapping in Huff model

In [0]:
att_field = "raw_visit_counts"
location_name = "Whole Foods Market"
pos = [0.717,0.805]
alpha = pos[0]
beta = pos[1]
mx_hf,cor = Corr(mx,alpha,beta,att_field,location_name)

In [0]:
cor

0.8643218199446577

In [0]:
mx_hf_wholefoods = mx_hf[mx_hf['location_name'] == "Whole Foods Market"]

In [0]:
list1 = mx_hf_wholefoods['actual_pro']
list2 = mx_hf_wholefoods['pro']

In [0]:
numpy.corrcoef(list1,list2)

array([[1.        , 0.86432182],
       [0.86432182, 1.        ]])

## The correlation result for the estimated hourly visit probability using the T-Huff model compared to the actual visit probability observed in SafeGraph POI visits.

In [0]:
att_field = "raw_visit_counts"
location_name = "Whole Foods Market"
pos = [0.787,0.765]
alpha = pos[0]
beta = pos[1]
mx_hf,cor = CorrHourlyLocation(mx,alpha,beta,att_field,location_name,"popularity_hourly_pro",'pro_hourly_actual','pro_hourly_est')

In [0]:
cor

0.8897300294403433